# COM6509 Resit Assignment 2022 - Part 1 Brief 

## Deadline: As specified in Blackboard

## Scope: Sessions 1 to 5

## Number of marks available for Part 1: 50

### Please READ the whole assignment first, before starting to work on it.

### How and what to submit

A. A **Jupyter Notebook** with the code in all the cells executed and outputs displayed.

B. Name your Notebook **COM6509_ResitAssignment_Part1_Username_XXXXXX.ipynb** where XXXXXX is your username such as such as abc18de.  

C. Upload a .zip file to Blackboard before the deadline that contains the Jupyter Notebook in B of this part and the Jupyter Notebook from Part 2. 

D. **NO DATA UPLOAD**: Please do not upload the data files used in this Notebook. We have a copy already. 


### Assessment Criteria 

* Being able to use numpy and pandas to preprocess a dataset.

* Being able to use numpy to build a machine learning pipeline for supervised learning. 

* Being able to follow the steps involved in an end-to-end project in machine learning.

* Being able to use scikit-learn to design a machine learning model pipeline

# A dataset of air quality

The dataset you will use in this assignment comes from a popular machine learning repository that hosts open source datasets for educational and research purposes, the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). We are going to use regularised ridge regression and random forests for predicting air quality. The description of the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Air+Quality).

In [1]:
import urllib.request
doq = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
pat_sav = "./AirQualityUCI.zip"
urllib.request.urlretrieve(doq, pat_sav)
#urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip")

('./AirQualityUCI.zip', <http.client.HTTPMessage at 0x1c66a88fbb0>)

In [2]:
import zipfile
zip = zipfile.ZipFile('./AirQualityUCI.zip', 'r')
for name in zip.namelist():
    zip.extract(name, '.')

In [3]:
# The .csv version of the file has some typing issues, so we use the excel version
import pandas as pd 
air_quality_full = pd.read_excel('./AirQualityUCI.xlsx', usecols=range(2,15))

We can see some of the rows in the dataset 

In [4]:
air_quality_full.sample(5)

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
7235,0.9,1004.75,-200,3.448699,679.25,88.8,874.50,66.8,1120.00,744.50,10.675,69.474998,0.891872
443,0.7,898.00,33,1.677902,559.25,55.0,1441.25,70.0,1179.75,648.25,10.125,51.200000,0.634077
2883,2.9,1268.25,-200,16.852308,1205.50,201.0,648.00,137.0,1899.50,1433.50,32.200,27.575000,1.304441
1752,2.8,1409.00,-200,13.873148,1112.50,157.0,710.00,110.0,1849.50,1255.50,25.150,40.124999,1.264180
1451,-200.0,867.00,-200,2.577932,624.75,-200.0,1212.50,-200.0,1385.00,652.25,14.025,63.824999,1.016724


The target variable corresponds to the CO(GT) variable of the first column. The following columns correspond to the variables in the feature vectors, *e.g.*, PT08.S1(CO) is $x_1$ up until AH which is $x_D$. The original dataset also has a date and a time columns that we are not going to use in this assignment.

### Removing instances 

The dataset has missing values tagged with a -200 value. To simplify the design of the machine learning models in this assignment, we perform the following two operations to the dataset right from the beginning:

* we will remove the rows for which the target variable has missing values. We are doing supervised learning so we need all our data observations to have known target values.

* we will remove features with more than 20% of missing values. 

The code below performs both operations

In [5]:
# We first remove the rows for which there are missing values in the target feature
air_quality = air_quality_full.loc[air_quality_full.iloc[:, 0]!=-200, :]

In [6]:
# We now remove the columns (features) for which there are more that 20% of missing values
import numpy as np
ndata, ncols = np.shape(air_quality) # number of data observations and number of columns in the dataframe
pmissing = np.empty(ncols)         # An empty vector that will keep the percentage of missing values per feature
for i in range(ncols):
    pmissing[i] = (air_quality.iloc[:, i]==-200).sum()/ndata # Computes the percentage of missing values per column
air_quality = air_quality.loc[:, pmissing < 0.2]

### Splitting the dataset 

Before designing any machine learning model, we need to set aside the test data. We will use the remaining training data for fitting the model. *It is important to remember that the test data has to be set aside before preprocessing*. 

Any preprocessing that you do has to be done only on the training data and several key statistics need to be saved for the test stage.  Separating the dataset into training and test before any preprocessing has happened help us to recreate the real world scenario where we will deploy our system and for which the data will come without any preprocessing.

Furthermore, we are going to use *hold-out validation* for validating our predictive model so we need to further separate the training data into a training set and a validation set.

We split the dataset into a training set, a validation set and a test set. The training set will have 70% of the total observations, the validation set 15% and the test set, the remaining 15%. For making the random selections of the training and validation sets **make sure that you use a random seed that corresponds to the last five digits of your student UCard**. In the code below, I have used 55555 as an example of my random seed.

In [7]:
np.random.seed(55555)                 # Make sure you use the last five digits of your student UCard as your seed
index = np.random.permutation(ndata)  # We permute the indexes 
N = np.int64(np.round(0.70*ndata))    # We compute N, the number of training instances
Nval = np.int64(np.round(0.15*ndata)) # We compute Nval, the number of validation instances   
Ntest = ndata - N - Nval              # We compute Ntest, the number of test instances
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data

The assigment is divided into two sections. In the **first section**, you will design a regularised ridge regression model trained with stochastic gradient descent. You will write all the code from scratch. You should not use any library that already implements any of the routines considered in this section, for example, scikit-learn. In the **second section**, you will design a random forests model and you are allowed to use scikit-learn in this section.

When writing your code, you will find out that there are operations that are repeated at least twice. We will assign marks for use of Python functions and for commenting your code. The marks will be assigned as:

* Did you include Python functions to solve the question and avoid repeating code? (**2 marks**)
* Did you comment your code to make it readable to others? (**2 marks**)

# 1. Using regularised ridge regression to predict air quality (20 marks)

**DO NOT USE scikit-learn or any other machine learning library for the questions on this section. You are meant to write Python code from scratch. You can use Pandas and Numpy. Using scikit-learn or any other machine learning library for the questions in this section will give ZERO marks. No excuse will be accepted.**

Regularisation is a technique commonly used in Machine Learning to prevent overfitting. It consists on adding terms to the objective function such that the optimisation procedure avoids solutions that just learn the training data. Popular techniques for regularisation in Supervised Learning include [Lasso Regression](https://en.wikipedia.org/wiki/Lasso_(statistics)), [Ridge Regression](https://en.wikipedia.org/wiki/Tikhonov_regularization) and the [Elastic Net](https://en.wikipedia.org/wiki/Elastic_net_regularization). 

In this part of the Assignment, you will be looking at Ridge Regression and implementing equations to optimise the objective function using the update rules for stochastic gradient descent. You will use those update rules for making predictions on the Air Quality dataset.

## 1.1 Ridge Regression

Let us start with a data set for training $\mathcal{D} = \{\mathbf{y}, \mathbf{X}\}$, where the vector $\mathbf{y}=[y_1, \cdots, y_N]^{\top}$ and $\mathbf{X}$ is the design matrix from Lab 4, this is, 

\begin{align*}
    \mathbf{X} = 
                \begin{bmatrix}
                        1 & x_{1,1} & \cdots & x_{1, D}\\
                        1 & x_{2,1} & \cdots & x_{2, D}\\
                   \vdots &  \vdots\\
                        1 & x_{N,1} & \cdots & x_{N, D}
                \end{bmatrix}
               = 
               \begin{bmatrix}
                      \mathbf{x}_1^{\top}\\
                       \mathbf{x}_2^{\top}\\
                          \vdots\\
                        \mathbf{x}_N^{\top}
                \end{bmatrix}.
\end{align*}

Our predictive model is going to be a linear model

$$ f(\mathbf{x}_i) = \mathbf{w}^{\top}\mathbf{x}_i,$$

where $\mathbf{w} = [w_0\; w_1\; \cdots \; w_D]^{\top}$.

The **objective function** we are going to use has the following form

$$ E(\mathbf{w}, \lambda) = \frac{1}{N}\sum_{n=1}^N (y_n - f(\mathbf{x}_n))^2 + \frac{\lambda}{2}\sum_{j=0}^D w_j^2,$$

where $\lambda>0$ is known as the *regularisation* parameter.

This objective function was studied in Lecture 4. 

The first term on the rhs is what we call the "fitting" term whereas the second term in the expression is the regularisation term. Given $\lambda$, the two terms in the expression have different purposes. The first term is looking for a value of $\mathbf{w}$ that leads the squared-errors to zero. While doing this, $\mathbf{w}$ can take any value and lead to a solution that it is only good for the training data but perhaps not for the test data. The second term is regularising the behavior of the first term by driving the $\mathbf{w}$ towards zero. By doing this, it restricts the possible set of values that $\mathbf{w}$ might take according to the first term. The value that we use for $\lambda$ will allow a compromise between a value of $\mathbf{w}$ that exactly fits the data (first term) or a value of $\mathbf{w}$ that does not grow too much (second term).

This type of regularisation has different names: ridge regression, Tikhonov regularisation or $\ell_2$ norm regularisation. 

## 1.2 Optimising the objective function with respect to $\mathbf{w}$

There are two ways we can optimise the objective function with respect to $\mathbf{w}$. The first one leads to a closed form expression for $\mathbf{w}$ and the second one using an iterative optimisation procedure that updates the value of $\mathbf{w}$ at each iteration by using the gradient of the objective function with respect to $\mathbf{w}$,
$$
\mathbf{w}_{\text{new}} = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}},
$$
where $\eta$ is the *learning rate* parameter and $\frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}}$ is the gradient of the objective function.

It can be shown (this is a question in the Exercise Sheet 4) that a closed-form expression for the optimal $\mathbf{w}_*$ is given as

\begin{align*}            
            \mathbf{w}_*& = \left(\mathbf{X}^{\top}\mathbf{X} + \frac{\lambda N}   
                                     {2}\mathbf{I}\right)^{-1}\mathbf{X}^{\top}\mathbf{y}.
\end{align*}

Alternatively, we can find an update equation for $\mathbf{w}_{\text{new}}$ using gradient descent leading to:

\begin{align*}
   \mathbf{w}_{\text{new}} & = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}
                              {d\mathbf{w}},\\
                           & = \mathbf{w}_{\text{old}} +  \frac{2\eta}{N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n  
                       - \eta\lambda\mathbf{w}_{\text{old}}\\
                           & = (1 - \eta\lambda)\mathbf{w}_{\text{old}} + \frac{2\eta}
                               {N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n
\end{align*}

## 1.3 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 1.a: Missing values and standardisation (4 marks)

* For all the other features with missing values, use the mean value of the non-missing values for that feature to perform imputation. Save these mean values, you will need them when performing the validation stage (**2 marks**).

* Once you have imputed the missing data, we need to standardise the input vectors. Standardise the training data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Keep the mean values and standard deviations, you will need them at validation time (**2 marks**).

#### Question 1.a Answer

In [8]:
# Write your code here
# first, use na value to replace missing values so the mean would not be affected
training_data = data_training_unproc.replace(-200, np.NaN)
# save mean values and standard deviation values
training_data_mean = training_data.mean()
training_data_std = training_data.std()
# replace missing values by mean
training_data = training_data.fillna(training_data_mean)

# standardize function
def standardize(data, mean, std):
    return (data-mean)/std
# standardize
training_data = standardize(data=training_data, mean=training_data_mean, std=training_data_std)

## 1.4 Training and validation stages

We have now curated our training data by removing data observations and features with a large amount of missing values. We have also normalised the feature vectors. We are now in a good position to work on developing the prediction model and validating it. We will use gradient descent for iterative optimisation. 

We first organise the dataframe into the vector of targets $\mathbf{y}$, call it `yTrain`, and the design matrix $\mathbf{X}$, call it `XTrain`.

In [9]:
# Write your code here
yTrain = training_data['CO(GT)'].to_numpy()
XTrain = training_data.iloc[:,1:13].to_numpy()
# add intercept to features
def add_intercept(data):
    intercept = np.ones(data.shape[0]).reshape(-1,1)
    result = np.column_stack((intercept, data))
    return result
XTrain = add_intercept(XTrain)

### Question 1.b: finding the optimal $\mathbf{w}$ with stochastic gradient descent (6 marks)

Use gradient descent to iteratively compute the value of $\mathbf{w}_{\text{new}}$. Instead of using all the training set in `XTrain` and `yTrain` to compute the gradient, use a subset of $S$ instances in `XTrain` and `yTrain`. This is sometimes called *minibatch gradient descent* where $S$ is the size of the minibacth. When using gradient descent with minibatches, you need to find the best values for three parameters: $\eta$, the learning rate, $S$, the number of datapoints in the minibatch and $\gamma$, the regularisation parameter.

* In this question we will use the validation data. So before proceeding to the next steps, make sure that you:  replace the missing values on each feature variables with the mean value you computed with the training data; standardise the validation data using the means and standard deviations computed from the training data (**2 marks**).
    
* Create a grid of values for the parameters $\gamma$ and $\eta$ using `np.logspace` and a grid of values for $S$ using `np.linspace`. Because you need to find three parameters, start with five values for each parameter in the grid and see if you can increase it. Make sure you understand what is the meaning of `np.logspace` and `np.linspace`. Notice that you can use negative values for `start` in `np.logspace` (**2 marks**).

* For each value that you have of $\gamma$, $\eta$ and $S$ from the previous step, use the training set to compute $\mathbf{w}$ using minibatch gradient descent and then measure the RMSE over the validation data. For the validation data, make sure you preprocess it before applying the prediction model over it. For the minibatch gradient descent choose to stop the iterative procedure after $200$ iterations (**2 marks**).

* Choose the values of $\gamma$, $\eta$ and $S$ that lead to the lowest RMSE and save them. You will use them at the test stage.

#### Question 1.b Answer

In [10]:
# Write your code here
# standardize validation data
validation_data = data_val_unproc.replace(-200, training_data_mean)
validation_data = standardize(data=validation_data, mean=training_data_mean, std=training_data_std)
# separate validation data into x and y, and then add intercept
yVal = validation_data['CO(GT)'].to_numpy().reshape(-1,1)
XVal = validation_data.iloc[:,1:13].to_numpy()
XVal = add_intercept(XVal)

# create learning rate, regularisation strength, and batch size
learning_rate_vec = np.logspace(-4, -1, 4)
regularisation_strength_vec = np.logspace(-3, -1, 4)
batch_size_vec = np.linspace(100, 1000, 4)

# create minibatches
def get_minibatch(x, y, batch_size):
    minibatch = []
    # shuffle the data
    combined_data = np.column_stack((y, x))
    np.random.shuffle(combined_data)
    # calculate the amount of batches
    batches = len(x)//batch_size
    # get the batches with full batch size
    for i in range(batches):
        batch = combined_data[batch_size*i:batch_size*(i+1)]
        minibatch.append((batch[:,0], batch[:,1:combined_data.shape[1]]))
    # add the remaining data to last batch
    if len(x)%batch_size != 0:
        batch = combined_data[batch_size*batches:len(x)]
        minibatch.append((batch[:,0], batch[:,1:combined_data.shape[1]]))
    return minibatch

# create predictions
def predict(weight, x):
    result = np.dot(x, weight)
    return result

# create function to calculate rmse after mini batch gradient descent
def minibatch_gradient_descent(xtrain, ytrain, xval, yval, lr, regularisation_strength, batchsize, epochs=200):
    # initialize weights
    weights = np.random.normal(size = [xtrain.shape[1] ,1])
    for i in range(epochs):
        # in each epoch, create mini batches
        minibatches = get_minibatch(xtrain, ytrain, int(batchsize))
        for j in range(len(minibatches)):
            # train model with minibatch features x, true y
            x = minibatches[j][1]
            y = minibatches[j][0].reshape(-1,1)
            
            y_predict = predict(weights, x)
            loss = y - y_predict
            gradient = -(2/batchsize)*(x.T@loss) + regularisation_strength*weights # calculate gradient
            weights = weights - lr*gradient # update weights
    val_prediction = predict(weights, xval)
    val_loss = yval - val_prediction
#     val_rmse = np.sum(val_loss ** 2)
    val_rmse = np.sqrt(np.mean(val_loss ** 2))
    return val_rmse

rmse_min = 99999
# train the model with different lr, batch size, regularisation strength
for i in range(len(learning_rate_vec)):
    for j in range(len(regularisation_strength_vec)):
        for k in range(len(batch_size_vec)):
            rmse = minibatch_gradient_descent(xtrain=XTrain, ytrain=yTrain, xval=XVal, yval=yVal, lr=learning_rate_vec[i], regularisation_strength=regularisation_strength_vec[j], batchsize=batch_size_vec[k])
            if rmse<rmse_min:
                rmse_min = rmse
                lr_min = learning_rate_vec[i]
                rs_min = regularisation_strength_vec[j]
                bs_min = batch_size_vec[k]

print('lowest rmse: ', rmse_min, 'lr: ', lr_min, 'regularisation strength: ', rs_min, 'batch size: ', bs_min)

lowest rmse:  0.3555876451576074 lr:  0.01 regularisation strength:  0.021544346900318832 batch size:  400.0


## 1.5 Test stage 

We now know which one is the best model, according to the validation data. We will now put together the training data and the validation data, perform the preprocessing as we did before, this is, treat the missing values and standardise the inputs. We will train the model again using the minibatch stochastic gradient descent and finally compute the RMSE over the test data.


### Question 1.c: combine the original training and original validation data and perform the preprocessing again to this new data (4 marks)

Put together the original training and validation dataset and perform the same preprocessing steps than before, these are: 

* for each feature, impute the missing values with the mean values of the non-missing values (**2 marks**) 

* stardardise the new training set (**2 marks**).

#### Question 1.c Answer

In [11]:
# Write your code here
# combine training and validating data
whole_train = pd.concat([data_training_unproc, data_val_unproc])
whole_train = whole_train.replace(-200, np.NaN)
whole_train = whole_train.fillna(whole_train.mean())
whole_train = standardize(data=whole_train, mean=whole_train.mean(), std=whole_train.std())

yTrain_final = whole_train['CO(GT)'].to_numpy()
XTrain_final = whole_train.iloc[:,1:13].to_numpy()
XTrain_final = add_intercept(XTrain_final)

### Question 1.d: Preprocess the test data, train the model and predict over the test data (6 marks)

Preprocess the test data and train a new model using the new training set. Finally, report the RMSE over the test set:

* Preprocess the test data by imputing the missing data and standardising it (**2 marks**). 

* Use the best values of $\gamma$, $\eta$ and $S$ found in the validation set and train a new regularised linear model with stochastic gradient descent (**2 marks**).

* Report the RMSE over the test data (**2 marks**).

#### Question 1.d Answer

In [12]:
# Write your code here
testing_data = data_test_unproc.replace(-200, np.NaN)
testing_data = testing_data.fillna(testing_data.mean())
testing_data = standardize(data=testing_data, mean=testing_data.mean(), std=testing_data.std())

yTest = testing_data['CO(GT)'].to_numpy().reshape(-1,1)
XTest = testing_data.iloc[:,1:13].to_numpy()
XTest = add_intercept(XTest)

final_rmse = minibatch_gradient_descent(XTrain_final, yTrain_final, XTest, yTest, lr_min, rs_min, bs_min)
print('RMSE over the test data: ', final_rmse)

RMSE over the test data:  0.3226153942744631


# 2. Random forests (26 marks)

**USE scikit-learn for the questions on this section.**

In section 1, you used a regularised ridge regression model trained with SGD to create a linear predictive model. In this part of the assignment, you will use **scikit-learn** to train a random forest for regression over the air quality dataset.

## 2.1 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 2.a: Pipeline for missing values and standardisation (6 marks)

* Employ the `SimpleImputer` method in Scikit-learn to impute the missing values in each column using the mean value of the non-missing values, instead (**2 marks**).

* Standardise the data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Employ the `StandardScaler` method (**2 marks**).

* Create a `Pipeline` that you can use to preprocess the data by filling missing values and then standardising the features (**2 marks**).

#### Question 2.a Answer

In [13]:
# Write your code here
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

processing_data = Pipeline([("imputer", SimpleImputer(missing_values = -200, strategy = "mean")), ("scaler", StandardScaler())])

### Question 2.b: Use the Pipeline to fit the training data and transform the validation data (4 marks)

In the previous question, you created a `Pipeline` for applying a `SimpleImputer` and a `StandardScaler`. Use the Pipeline to fit the training data (**2 marks**) and transform the validation data (**2 marks**).

#### Question 2.b Answer

In [14]:
# Write your code here
yTrain = data_training_unproc['CO(GT)'].to_numpy().reshape(-1,1)
XTrain = data_training_unproc.iloc[:,1:13].to_numpy()

yVal = data_val_unproc['CO(GT)'].to_numpy().reshape(-1,1)
XVal = data_val_unproc.iloc[:,1:13].to_numpy()

XTrain = processing_data.fit_transform(XTrain)
XVal = processing_data.transform(XVal)

## 2.2 Random forest to predict air quality 

We now use random forests to predict air quality. Remember that the tree ensemble in random forests is built by training individual regression trees on different subsets of the training data and using a subset of the available features. For regression, the prediction is the average of the individual predictions of each tree. Some of the parameters required in the Random Forest implementation in Scikit-learn include:

Some of the additional parameters required in the Random Forest implementation in Scikit-learn include

> **n_estimators** the total number of trees to train<p>
**max_features** number of features to use as candidates for splitting at each tree node. <p>
    **boostrap**: Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.<p>
   **max_samples**: If bootstrap is True, the number of samples to draw from X to train each base estimator.

### Question 2.c: train a random forest (8 marks)

In this question, you will train a random forest for predicting over the validation data. Use cross-validation over the validation data to select the best set of paramaters for the random forest regressor. Parameters to include in your exploration are **n_estimators**, **max_features** and **max_samples**. Use `np.linspace` or `np.logspace` to define ranges of values to explore for each parameter and create a grid to be assessed over the validation data. Make sure you use the same validation data that was given to you.

* Use `PredefinedSplit` to tell the cross validator which instances to use for training and which ones for validation (**2 marks**).

* Create a grid of values to explore that include a range of at least five values for each parameter **n_estimators**, **max_features** and **max_samples** (**2 marks**). 

* Train a random forest for regression model that uses the grid of parameters you created before. Use `GridSearchCV` to find the best set of parameters by performing cross-validation over the predefined split. (**2 marks**).

* Print the best values in your grid for **n_estimators**, **max_features** and **max_samples** (**2 marks**).

#### Question 2.c Answer

In [15]:
# Write your code here
from sklearn.model_selection import PredefinedSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# create predefined split
test_fold = np.ones((XTrain.shape[0]+XVal.shape[0]))
test_fold[0:XTrain.shape[0]] = 0
ps = PredefinedSplit(test_fold)

# create n_estimators, max_features, max_samples vectors
n_estimators = (np.linspace(5, 50, 5)).astype(int)
max_features = (np.linspace(1, 10, 5)).astype(int)
max_samples = np.logspace(-2, 0, 5, endpoint=False)

# create grid of parameters
param_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_samples': max_samples,
}

# get the input x and y
x = np.vstack((XTrain, XVal))
y = np.vstack((yTrain, yVal))

# perform gridsearchcv
random_forest = GridSearchCV(estimator=RandomForestRegressor(), param_grid=param_grid, cv=ps)
random_forest.fit(x, y.ravel())

print(random_forest.best_estimator_)

RandomForestRegressor(max_features=3, max_samples=0.3981071705534973,
                      n_estimators=27)


### Question 2.d: train a new model over the whole training data and report the prediction over the test set (8 marks)


Now that we have identified the best paramaters of the regression model, we use these parameters to train a new model over the whole training data (`data_training` plus `data_val`). We apply this model to the test set and report the performance.

* Create a new preprocessing pipeline for taking care of the missing values and standardisation over the whole training data (**2 marks**).

* Apply the created preprocessing pipeline to the test data (**2 marks**).

* Fit a random forest regression model to the training data using the best parameters found in Question 2.c (**2 marks**).

* Compute the RMSE over the test data and report the result (**2 marks**).

#### Question 2.d Answer

In [16]:
# Write your code here
# concatenate val and train
whole_train = pd.concat([data_training_unproc, data_val_unproc])
yTrain_final = whole_train['CO(GT)'].to_numpy().reshape(-1, 1)
XTrain_final = whole_train.iloc[:,1:13].to_numpy()

yTest = data_test_unproc['CO(GT)'].to_numpy()
XTest = data_test_unproc.iloc[:,1:13].to_numpy()

# preprocessing
XTrain_final = processing_data.fit_transform(XTrain_final)
XTest = processing_data.transform(XTest)

# assign the hyperparameters in randomforest regression model
best_randomforest = RandomForestRegressor(n_estimators=random_forest.best_params_['n_estimators'],
                                          max_features=random_forest.best_params_['max_features'], 
                                          max_samples=random_forest.best_params_['max_samples'])
# train the model
model = best_randomforest.fit(XTrain_final, yTrain_final.ravel())

# prediction on XTest
predictions = model.predict(XTest)
loss = yTest - predictions
rmse = np.sqrt(np.mean(loss ** 2))
print('RMSE over the test data: ', rmse)

RMSE over the test data:  0.4185427662843116
